# 



Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT

In [2]:
from __future__ import print_function, division

import thinkstats2
import thinkplot

import pandas as pd
import numpy as np

from fractions import Fraction

%matplotlib inline

In [13]:
def dot(x, y):
    x = np.asarray(x)
    y = np.asarray(y)
    return np.sum(x * y)

In [17]:
dot([1,2,3], (4,5,6))

32

In [26]:
class Matrix:
    def __init__(self, array):
        self.array = np.asarray(array)
        
    def __add__(self, other):
        return Matrix(self.array + other.array)
    
    def __sub__(self, other):
        return Matrix(self.array - other.array)
    
    def __mul__(self, other):
        return Matrix(self.array.dot(other.array))
    
    def __truediv__(self, other):
        return Matrix(np.linalg.solve(self.array, other.array))
    
    def __str__(self):
        return '\n'.join(str(row) for row in self)
    
    def __repr__(self):
        return 'Matrix(%s)' % str(self.array)
    
    def __len__(self):
        return len(self.array)
    
    def __getitem__(self, index):
        return self.array[index]
    
    def __setitem__(self, index, row):
        self.array[index] = row

In [27]:
def random_array(*shape):
    return np.random.randint(1, 10, shape)

In [28]:
A = Matrix(random_array(3, 3))
print(A)

[3 1 6]
[6 6 7]
[1 1 7]


In [29]:
len(A)

3

In [30]:
for row in A:
    print(row)

[3 1 6]
[6 6 7]
[1 1 7]


In [31]:
B = Matrix(random_array(3, 3))
print(B)

[2 1 8]
[1 5 7]
[1 2 9]


In [32]:
print(A + B)

[ 5  2 14]
[ 7 11 14]
[ 2  3 16]


In [33]:
print(A - B)

[ 1  0 -2]
[5 1 0]
[ 0 -1 -2]


In [34]:
print(A * B)

[13 20 85]
[ 25  50 153]
[10 20 78]


In [43]:
x = Matrix(random_array(3))
print(x)

4
1
1


In [61]:
def matrix_vector_product(A, x):
    array = np.zeros(len(A), dtype=x.array.dtype)
    for i, row in enumerate(A):
        array[i] = dot(row, x)
    return Matrix(array)

In [62]:
print(matrix_vector_product(A, x))

19
37
12


In [45]:
print(A * x)

19
37
12


In [188]:
b = A * x
print(b)

[[100]
 [100]
 [ 66]]


In [189]:
print(A / b)

[[ 9.]
 [ 7.]
 [ 1.]]


In [190]:
A.array.shape

(3, 3)

In [191]:
b.array.shape

(3, 1)

In [231]:
m = np.hstack([A.array, b.array]).astype(Fraction)
print(m)

[[7 4 9 100]
 [9 2 5 100]
 [4 4 2 66]]


In [232]:
m[1] -= m[0]
print(m)

[[7 4 9 100]
 [2 -2 -4 0]
 [4 4 2 66]]


In [233]:
m[:, :-1]

array([[7, 4, 9],
       [2, -2, -4],
       [4, 4, 2]], dtype=object)

In [234]:
m[:, -1]

array([100, 0, 66], dtype=object)

In [235]:
def solve_augmented(m):
    m = m.astype(float)
    return np.linalg.solve(m[:, :-1], m[:,-1])

In [236]:
print(solve_augmented(m))

[ 9.  7.  1.]


In [237]:
row1 = 0
row2 = 1
col = 0
pivot = m[row1, col]
victim = m[row2, col]
m[row1], pivot, victim, m[row1] * Fraction(victim, pivot)

(array([7, 4, 9, 100], dtype=object),
 7,
 2,
 array([Fraction(2, 1), Fraction(8, 7), Fraction(18, 7), Fraction(200, 7)], dtype=object))

In [238]:
m[row2] -= m[row1] * Fraction(victim, pivot)
print(m)

[[7 4 9 100]
 [Fraction(0, 1) Fraction(-22, 7) Fraction(-46, 7) Fraction(-200, 7)]
 [4 4 2 66]]


In [241]:
def clobber(m, row1, row2, col):
    pivot = m[row1, col]
    victim = m[row2, col]
    m[row2] -= m[row1] * Fraction(victim, pivot)

In [242]:
clobber(m, 0, 2, 0)
print(m)

[[7 4 9 100]
 [Fraction(0, 1) Fraction(-22, 7) Fraction(-46, 7) Fraction(-200, 7)]
 [Fraction(0, 1) Fraction(12, 7) Fraction(-22, 7) Fraction(62, 7)]]


In [244]:
clobber(m, 1, 2, 1)
print(m)

[[7 4 9 100]
 [Fraction(0, 1) Fraction(-22, 7) Fraction(-46, 7) Fraction(-200, 7)]
 [Fraction(0, 1) Fraction(0, 1) Fraction(-74, 11) Fraction(-74, 11)]]
